In [ ]:

Secret_NGROK = 'NGROK_API_KEY' #@param {type: "string"}
Secret_CIVITAI = 'Civitai_API_KEY' #@param {type: "string"}
#@markdown ### ▶️ CivitAI API Key (Required for certain models):
Civitai_API_KEY = '' #@param {type: "string"}

NGROK_API_KEY='2YhpTDqMRRtSe7c22Jrl4M8pvYW_6efmtGHdorSmkT8YVQUKC' #@param {type: "string"}
use_local_storage = True #@param {type:"boolean"}
output_path = 'COMFY' #@param {type:"string"}
version='' #@param {type: "string"}

Flux1_dev = False #@param{type: "boolean"}


Anything_XL = True #@param{type: "boolean"}


Custom_LoRA_URL = "" #@param {type: "string"}
Custom_LoRA_Name = "" #@param {type: "string"}
Custom_Model_URL = "" #@param {type: "string"}
Custom_Model_Name = "" #@param {type: "string"}


SD_1_5_ControlNet_models = False #@param{type: "boolean"}
SDXL_ControlNet_models = False #@param{type: "boolean"}
IP_Adapter_models = False #@param{type: "boolean"}


clip_l = False #@param{type: "boolean"}
t5xxl = False #@param{type: "boolean"}
clip_vision_g = False #@param{type: "boolean"}
google_t5_v1 = False #@param{type: "boolean"}
llava_llama3_fp16 = False #@param{type: "boolean"}
llava_llama3_fp8 = False #@param{type: "boolean"}

#@markdown ### ▶️ ComfyUI Nodes:
comfyui_controlnet_aux = False #@param{type: "boolean"}
comfyui_inpaint_nodes = False #@param{type: "boolean"}
x_flux_comfyui = True #@param{type: "boolean"}
ComfyUI_IPAdapter_plus = True #@param{type: "boolean"}
ComfyUI_AnimateDiff_Evolved = False #@param{type: "boolean"}
civitai_comfy_nodes = True #@param{type: "boolean"}

sdxl_prompt_styler = True #@param{type: "boolean"}
comfy_image_saver = True #@param{type: "boolean"}
ComfyUI_Crystools = True #@param{type: "boolean"}

ComfyUI_Advanced_ControlNet = False #@param{type: "boolean"}
ComfyUI_Frame_Interpolation = False #@param{type: "boolean"}
Derfuu_ComfyUI_ModdedNodes = True #@param{type: "boolean"}


LoadLoraWithTags = True #@param{type: "boolean"}


Extra_arguments = '--disable-smart-memory --port 8188' #@param {type: "string"}

Clear_Log = True #@param{type: "boolean"}

#####################
import subprocess
import threading
import time
import socket
import urllib.request
import os
import glob

def link_files(source, dest):
  if not os.path.exists(source):
    !mkdir -p {source}
  if not os.path.exists(dest):
    !mkdir -p {dest}
  model_files = glob.glob(source + '/*')
  %cd {dest}
  for f in model_files:
    print(f'Linking model {f} in {dest}')
    !ln -s {f}

def initStorage(output_path, app_local_path):
  if use_local_storage:
    # Use local storage
    app_storage_path = "/content/local_storage"
    models_storage_path = f"{app_storage_path}/models"
    image_output_path = f"{app_storage_path}/outputs"
    image_input_path = f"{app_storage_path}/inputs"

    # Create local directories
    !mkdir -p {app_storage_path}
    !mkdir -p {models_storage_path}
    !mkdir -p {image_output_path}
    !mkdir -p {image_input_path}
  else:
    # Use Google Drive storage
    from google.colab import drive
    drive.mount('/content/drive')

    app_storage_path = f"/content/drive/MyDrive/{output_path}"
    models_storage_path = f"{app_storage_path}/models"
    image_output_path = f"{app_storage_path}/outputs"
    image_input_path = f"{app_storage_path}/inputs"

    # Create Google Drive directories
    !mkdir -p {app_storage_path}
    !mkdir -p {models_storage_path}
    !mkdir -p {image_output_path}
    !mkdir -p {image_input_path}

  # Set up model directories
  model_path_map = {
      f'{models_storage_path}/embeddings': f'{app_local_path}/models/embeddings',
      f'{models_storage_path}/animatediff': f'{app_local_path}/models/animatediff_models',
      f'{models_storage_path}/clip': f'{app_local_path}/models/clip',
      f'{models_storage_path}/clip_vision': f'{app_local_path}/models/clip_vision',
      f'{models_storage_path}/ControlNet': f'{app_local_path}/models/controlnet',
      f'{models_storage_path}/ESRGAN': f'{app_local_path}/models/upscale_models',
      f'{models_storage_path}/hypernetworks': f'{app_local_path}/models/hypernetworks',
      f'{models_storage_path}/insightface': f'{app_local_path}/models/insightface',
      f'{models_storage_path}/instantid': f'{app_local_path}/models/instantid',
      f'{models_storage_path}/layer_model': f'{app_local_path}/models/layer_model',
      f'{models_storage_path}/LDSR': f'{app_local_path}/models/LDSR',
      f'{models_storage_path}/liveportrait': f'{app_local_path}/models/liveportrait',
      f'{models_storage_path}/Lora': f'{app_local_path}/models/loras',
      f'{models_storage_path}/reactor': f'{app_local_path}/models/reactor',
      f'{models_storage_path}/Stable-diffusion': f'{app_local_path}/models/checkpoints',
      f'{models_storage_path}/unet': f'{app_local_path}/models/unet',
      f'{models_storage_path}/VAE': f'{app_local_path}/models/vae',
      f'{models_storage_path}/VAE-approx': f'{app_local_path}/models/vae',
      f'{models_storage_path}/xlabs/loras': f'{app_local_path}/models/xlabs/loras',
      f'{models_storage_path}/ultralytics/bbox': f'{app_local_path}/models/ultralytics/bbox',
      f'{models_storage_path}/ultralytics/segm': f'{app_local_path}/models/ultralytics/segm',
  }

  # Create symbolic links for all models
  for model_path in model_path_map:
    link_files(model_path, model_path_map[model_path])

  return image_output_path, image_input_path

def iframe_thread(port):
  while True:
      time.sleep(1)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("Tunnel Password:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')

def clear():
    from IPython.display import clear_output; return clear_output()

def downloadModel(url, rename=None, token=None):
    # Hugging Face model download
    if 'huggingface.co' in url:
        if rename:
            filename = rename
        else:
            filename = url.split('/')[-1].split('?')[0]  # Remove query parameters
        print(f"Downloading Hugging Face model: {filename}")
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -o {filename}

    # Civitai model download
    elif 'civitai.com' in url:
        filename = rename or url.split('/')[-1].split('?')[0]
        url_with_token = f"{url}?token={token}" if token else url
        print(f"Downloading Civitai model: {filename}")
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url_with_token} -o {filename}
    else:
        print("Unsupported URL format.")

def install_dependencies(repo_name):
  requirements_path = f"/content/ComfyUI/custom_nodes/{repo_name}/requirements.txt"
  if os.path.exists(requirements_path):
      print(f"Installing dependencies for {repo_name}...")
      subprocess.run(['pip', 'install', '-r', requirements_path], check=True)
  else:
      print(f"No requirements.txt found for {repo_name}. Skipping dependency installation.")

def install_custom_node(url):
  %cd /content/ComfyUI/custom_nodes
  repo_name = url.split('/')[-1]  # Extract repo name from the URL
  !git clone {url} --recursive
  install_dependencies(repo_name)

def install_comfyui_impact():
    %cd /content/ComfyUI/custom_nodes

    # Clone the repository manually
    !git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack.git

    # Change directory to the cloned repository
    %cd ComfyUI-Impact-Pack

    # Install dependencies from requirements.txt
    print("Installing dependencies from requirements.txt...")
    !pip install -r requirements.txt

    # Install the 'ultralytics' library, which is missing and needed for YOLO
    print("Installing ultralytics for YOLO...")
    !pip install ultralytics

    # Run the install.py script to complete the installation
    print("Running install.py...")
    !python install.py

def install_custom_nodes():
  %cd {customNodeRoot}
  install_custom_node('https://github.com/ltdrdata/ComfyUI-Manager')
  install_custom_node('https://github.com/ltdrdata/ComfyUI-Impact-Pack')
  install_custom_node('https://github.com/rgthree/rgthree-comfy')
  install_custom_node('https://github.com/WASasquatch/was-node-suite-comfyui')
  install_custom_node('https://github.com/pythongosssss/ComfyUI-Custom-Scripts')
  install_custom_node('https://github.com/ltdrdata/ComfyUI-Inspire-Pack')
  install_custom_node('https://github.com/jags111/efficiency-nodes-comfyui')
  install_custom_node('https://github.com/ssitu/ComfyUI_UltimateSDUpscale')
  install_custom_node('https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes')
  install_custom_node('https://github.com/pythongosssss/ComfyUI-WD14-Tagger')
  install_custom_node('https://github.com/cubiq/ComfyUI_essentials')
  install_custom_node('https://github.com/ltdrdata/ComfyUI-Impact-Subpack')

  if(comfyui_controlnet_aux):
    install_custom_node('https://github.com/Fannovel16/comfyui_controlnet_aux')
  if(comfyui_inpaint_nodes):
    install_custom_node('https://github.com/Acly/comfyui-inpaint-nodes')
  if(x_flux_comfyui):
    install_custom_node('https://github.com/XLabs-AI/x-flux-comfyui')
  if(ComfyUI_IPAdapter_plus):
    install_custom_node('https://github.com/cubiq/ComfyUI_IPAdapter_plus')
  if(civitai_comfy_nodes):
    install_custom_node('https://github.com/civitai/civitai_comfy_nodes')
  if(sdxl_prompt_styler):
    install_custom_node('https://github.com/twri/sdxl_prompt_styler')
  if(comfy_image_saver):
    install_custom_node('https://github.com/giriss/comfy-image-saver')
  if(ComfyUI_Crystools):
    install_custom_node('https://github.com/crystian/ComfyUI-Crystools')

  if(ComfyUI_Advanced_ControlNet):
    install_custom_node('https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet')

  if(Derfuu_ComfyUI_ModdedNodes):
    install_custom_node('https://github.com/Derfuu/Derfuu_ComfyUI_ModdedNodes')




  if(LoadLoraWithTags):
    install_custom_node('https://github.com/Extraltodeus/LoadLoraWithTags')

  install_comfyui_impact()

def install_Models():
  %cd {ComfyUIRoot}/models/checkpoints
  print('⏱ Downloading models ...')
  # Flux
  if Flux1_dev:
    downloadModel('https://huggingface.co/Comfy-Org/flux1-dev/resolve/main/flux1-dev-fp8.safetensors')

  if Anything_XL:
    downloadModel('https://civitai.com/api/download/models/384264',rename="Anything_XL.safetensors",token=Civitai_API_KEY)


  if Custom_Model_URL and Custom_Model_Name:
    print(f"Downloading custom model: {Custom_Model_Name}")
    downloadModel(Custom_Model_URL, rename=Custom_Model_Name, token=Civitai_API_KEY)


def install_LoRAModels():
  %cd {ComfyUIRoot}/models/loras

  if Custom_LoRA_URL and Custom_LoRA_Name:
    print(f"Downloading custom LoRA model: {Custom_LoRA_Name}")
    downloadModel(Custom_LoRA_URL, rename=Custom_LoRA_Name, token=Civitai_API_KEY)

def install_Embeddings():
  %cd {ComfyUIRoot}/models/embeddings
  downloadModel('https://civitai.com/api/download/models/9208',rename="easynegative.safetensors")
  downloadModel('https://civitai.com/api/download/models/930629',rename="deep_negative_pony.safetensors")

def install_VAE(vae_to_install):
  %cd {ComfyUIRoot}/models/vae
  downloadModel(str(vae_to_install))

def install_CLIP():
  %cd {ComfyUIRoot}/models/clip
  print(f"Downloading Clip Models")
  if(clip_l):
    downloadModel('https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors')
  if(t5xxl):
   downloadModel('https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors')
  if(clip_vision_g):
   downloadModel('https://huggingface.co/stabilityai/control-lora/resolve/main/revision/clip_vision_g.safetensors')
  if(google_t5_v1):
    downloadModel('https://huggingface.co/mcmonkey/google_t5-v1_1-xxl_encoderonly/resolve/main/t5xxl_fp8_e4m3fn.safetensors')
  if(llava_llama3_fp16):
    downloadModel('https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/llava_llama3_fp16.safetensors')
  if(llava_llama3_fp8):
    downloadModel('https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/llava_llama3_fp8_scaled.safetensors')

def install_Upscaler():
  %cd {ComfyUIRoot}/models/upscale_models
  downloadModel('https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth')
  downloadModel('https://huggingface.co/FacehugmanIII/4x_foolhardy_Remacri/resolve/main/4x_foolhardy_Remacri.pth')

def install_Ultralytics():
  %cd {ComfyUIRoot}/models/ultralytics/bbox
  downloadModel('https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8m.pt')
  downloadModel('https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8n.pt')

  downloadModel('https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov8s.pt')
  %cd {ComfyUIRoot}/models/ultralytics/segm
  downloadModel('https://huggingface.co/Bingsu/adetailer/resolve/main/person_yolov8m-seg.pt')


def install_ControlNetModels():
  %cd {ComfyUIRoot}/models/controlnet
  if SD_1_5_ControlNet_models:
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')
    downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors')
    downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/control_v1p_sd15_qrcode_monster_v2.safetensors')


  if SDXL_ControlNet_models:
    downloadModel('https://huggingface.co/xinsir/controlnet-openpose-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors', rename='diffusion_xl_openpose.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur_anime.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors')



def install_IPAdapter():
  if IP_Adapter_models:
    print('⏱ Downloading IPAdapters ...')
    !pip install insightface

    if not os.path.exists(f'{ComfyUIRoot}/models/ipadapter'):
      os.makedirs(f'{ComfyUIRoot}/models/ipadapter')

    %cd {ComfyUIRoot}/models/ipadapter
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light_v11.bin')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_vit-G.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.safetensors')

    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait-v11_sd15.bin')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl.bin')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait_sdxl.bin')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait_sdxl_unnorm.bin')
    %cd {ComfyUIRoot}/models/clip_vision
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors',rename="CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors")
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors',rename="CLIP-ViT-bigG-14-laion2B-39B-b160k.safetensors")

    %cd {ComfyUIRoot}/models/loras
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15_lora.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15_lora.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl_lora.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl_lora.safetensors')

def ngrok_thread(port):
  PASSWORD='googlecolab'
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  ngrok.set_auth_token(NGROK_API_KEY)
  http_tunnel = ngrok.connect(port, auth=f"{PASSWORD}:{PASSWORD}")
  print(f'Username/password: {PASSWORD}')
  print(f'ngrok public URL: {http_tunnel.public_url}')

#################################


!pkill -f 'python main.py'  # Kill any existing ComfyUI instances
!rm -rf /content/ComfyUI/logs/*  # Clear old logs

# Get secrets
from google.colab import userdata
try:
  NGROK_API_KEY = userdata.get(Secret_NGROK)
  print('Used NGROK secret.')
except:
  NGROK_API_KEY = None
try:
  Civitai_API_KEY = userdata.get(Secret_CIVITAI)
  print('Used Civitai secret.')
except:
  Civitai_API_KEY = None

# Define folders
root = '/content/'
ComfyUIRoot = os.path.join(root, 'ComfyUI')
customNodeRoot = os.path.join(ComfyUIRoot, 'custom_nodes')


# Install ComfyUI
!pip install --upgrade torch torchvision
%cd {root}
!git clone https://github.com/comfyanonymous/ComfyUI.git
if version:
  !git checkout {version}
%cd {ComfyUIRoot}
!pip install -r requirements.txt

# Initialize storage and get paths
image_output_path, image_input_path = initStorage(output_path, ComfyUIRoot)


!apt-get -y install -qq aria2

# ngrok
if NGROK_API_KEY:
  %pip install pyngrok
  from pyngrok import ngrok

# Install models
install_Models()
install_LoRAModels()
install_Embeddings()
install_ControlNetModels()

install_Upscaler()
install_CLIP()
install_IPAdapter()

install_Ultralytics()

# Install custom nodes
install_custom_nodes()

# localtunnel
!npm install -g localtunnel
if Clear_Log:
  clear()
port = 8188
if NGROK_API_KEY:
  threading.Thread(target=ngrok_thread, daemon=True, args=(port,)).start()
else:
  # Local tunnel
  !npm install -g localtunnel
  threading.Thread(target=iframe_thread, daemon=True, args=(port,)).start()
%cd /content/ComfyUI
!python main.py --dont-print-server --output-directory {image_output_path} --input-directory {image_input_path} {Extra_arguments}
